## Aprendizado Supervisionado
### prof. Duncan
Prática com Support Vector Classifier e Iris

Predição de Setosa com PetalLength e PetalWidth

In [ ]:
# atualizar pacote
#!pip install --upgrade scikit-learn
# pacotes básicos
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# pacotes do sklearn para acesso a datasets, preparação, modelagem e avaliação
from sklearn import datasets
# pacote pipeline para combinar preparação e modelagem
from sklearn.pipeline import Pipeline, make_pipeline
# arsenal de preparação
from sklearn.preprocessing import MinMaxScaler # rescala em min-max
from sklearn.preprocessing import StandardScaler # padroniza features removendo média e
#     escalando para variância unitária. Também chamado de z-score
#
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import StratifiedKFold as skf
from sklearn.model_selection import GridSearchCV as gscv
from sklearn.svm import SVC
from sklearn.inspection import DecisionBoundaryDisplay as DBD
from sklearn.metrics import accuracy_score as acc_score
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import ConfusionMatrixDisplay as CMD
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

#pacotes para apoio a leitura e gravação de datasets
from pathlib import Path
import csv

#pacotes para visualização e formatação
import pprint
#import graphviz

In [ ]:
# carga de dados
iris = datasets.load_iris()
print(iris.DESCR)

In [ ]:
# função para colorir pontos de diferentes classes
def pltcolor(y, c1, c2):
    cols=[]
    for i in range(len(y)):
        if y[i]==1:
            cols.append(c1)
        else:
            cols.append(c2)
    return cols

In [ ]:
# separação em features e target
# lembrando que queremos predizer Setosa
X = iris.data[:,2:4]
# classe positiva = 1; classe negativa= -1
y = iris.target
y = np.where(y==0, 1, -1)
iris_entrada = pd.DataFrame({'petallength': X[:,0],
                             'petalwidth' : X[:,1],
                             'class': y})

In [ ]:
# distribuição das classes com petallength e petalwidth
cols=pltcolor(y,'red', 'blue')
plt.scatter(X[:,0],X[:,1], c=cols)
plt.show()

In [ ]:
# separação em treino e teste, e X e y, e reescala

treinoX, testeX, treino_y, teste_y = tts(X, y, test_size=0.20, stratify=y)

MMS = MinMaxScaler()
treino_X = MMS.fit_transform(treinoX)
teste_X = MMS.transform(testeX)

print(treino_X.shape)
print(treino_y.shape)
print(teste_X.shape)
print(teste_y.shape)
print(np.stack(np.unique(teste_y, return_counts=True), axis=1))

In [ ]:
# distribuição das classes com petallength e petalwidth
cols=pltcolor(treino_y,'red', 'blue')
plt.scatter(treino_X[:,0],treino_X[:,1], c=cols)
plt.show()

In [ ]:
# definição do protocolo experimental
# para tanto, são testados diferentes valores que podem ser usados pelos diferentes kernels
# valores tirados da documentação do sklearn
#C_range = np.logspace(-2, 3, 6)
C_range = [100]
gamma_range = np.logspace(-9, 1, 11)
degree_range = np.linspace(1,4,4)
param_grid = [
  {'C': C_range, 'class_weight':['balanced'], 'kernel': ['linear']},
#  {'C': C_range, 'class_weight':['balanced'], 'degree': degree_range, 'kernel': ['poly']}#,
#  {'C': C_range, 'class_weight':['balanced'], 'gamma': gamma_range, 'kernel': ['rbf']},
#  {'C': C_range, 'class_weight':['balanced'], 'gamma': gamma_range, 'kernel': ['sigmoid']},
]

In [ ]:
%%time
# execução do protoloco experimental
# no caso, são 4 kernels, 6 valores distintos de C, 11 de gamma e 4 de degree
# cross validation com 5 folds,
# mais de 5.000 experimentos
crossval = skf(n_splits=5, shuffle=True, random_state=0)
grid = gscv(SVC(), param_grid=param_grid, cv=crossval)
grid.fit(treino_X,treino_y)

In [ ]:
# captura da melhor configuração
C_best = grid.best_params_['C']
cw_best = grid.best_params_['class_weight']
#gamma_best = grid.best_params_['gamma']
#degree_best = grid.best_params_['degree']
kernel_best = grid.best_params_['kernel']
#print('C={}  class_weight={}  gamma={}  kernel={}'.format(C_best, cw_best,gamma_best, kernel_best))
#print('C={}  class_weight={}  kernel={}  degree={}'.format(C_best, cw_best, kernel_best, degree_best))
print('C={}  class_weight={}  kernel={}'.format(C_best, cw_best, kernel_best))


In [ ]:
%%time
# indução do modelo de classificação por máquina de vetores de suporte
modelo = SVC(random_state=0,
             kernel=kernel_best,
             C=C_best,
             #gamma=gamma_best,
             class_weight=cw_best,
             verbose=True  )
modelo.fit(treino_X, treino_y)
teste_pred_y = modelo.predict(teste_X)
acuracia = acc_score(teste_y, teste_pred_y)
resultado = cm(teste_y, teste_pred_y)
cm_display = CMD(resultado).plot()
print('Acuracia={:.3f}  Núm.Iterações={}'.format(acuracia, modelo.n_iter_))


In [ ]:
modelo.dual_coef_

In [ ]:
modelo.intercept_

In [ ]:
modelo.support_

In [ ]:
modelo.n_support_

In [ ]:
SVs = modelo.support_
SV = np.zeros((len(SVs),2))
SV0 = np.zeros((modelo.n_support_[0],2))
SV1 = np.zeros((modelo.n_support_[1],2))
SV_y = np.zeros(len(SVs))
SV_y0 = np.zeros(modelo.n_support_[0])
SV_y1 = np.zeros(modelo.n_support_[1])

for i in range(len(SVs)):
    SV[i] = treino_X[SVs[i],:]
    SV_y[i] = treino_y[SVs[i]]
    if treino_y[SVs[i]] == 1:
        SV1[i - modelo.n_support_[0]] = SV[i]
        SV_y1[i - modelo.n_support_[0]] = SV_y[i]
    else:
        SV0[i] = SV[i]
        SV_y0[i] = SV_y[i]
    print(i, SVs[i], treino_X[SVs[i],:], treino_y[SVs[i]])


In [ ]:
disp = DBD.from_estimator(modelo, SV, response_method='predict',
                          grid_resolution=1000,
                          plot_method='contour',#'contourf',
                          eps=0.7,
                        cmap=plt.cm.coolwarm,
                        xlabel=iris.feature_names[2],
                        ylabel=iris.feature_names[3]
                         )
cols=pltcolor(SV_y,'red','blue')
plt.scatter(SV[:,0],SV[:,1], c=cols)
cols2=pltcolor(teste_pred_y,'green','black')
plt.scatter(teste_X[:,0],teste_X[:,1],c=cols2 )
plt.rcParams["figure.figsize"] = (8,8)
plt.show()